In [8]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [9]:
df = pd.read_csv("datasets/covid_philippines.csv")

In [10]:
df.head()

,Unnamed: 0,video_id,video_title,description,video_dop,view_count,like_count,comment_count,channel_id,channel_title,channel_dop,sub_count,total_videos
0,0,m3P-bmt3Uqw,JN.1 COVID-19 subvariant causing spike in cases,'An infectious disease expert says the new COV...,2023-12-25,6495,45,18.0,UCj6spMO3ybZPobE0T5perHA,CNN Philippines,2015-03-16,1390000,22741
1,1,sYI97jv-pZg,"PH records 2,725 new COVID cases from Dec. 12 ...",'The World Tonight: The daily average of the P...,2023-12-18,1982,14,3.0,UCvi6hEzLM-Z_unKPSuuzKvg,ANC 24/7,2010-01-29,1500000,70077
2,2,3YFpjgIQqEo,WATCH: DOH Usec. Tayag on rise of COVID-19 cas...,'Dateline Philippines: Karmina Constantino tal...,2023-12-07,9354,80,17.0,UCvi6hEzLM-Z_unKPSuuzKvg,ANC 24/7,2010-01-29,1500000,70077
3,3,CEIrzjA8euQ,DOH urges public to mask up amid rise in COVID...,'Dateline Philippines: The Philippine Health D...,2023-12-13,2010,26,2.0,UCvi6hEzLM-Z_unKPSuuzKvg,ANC 24/7,2010-01-29,1500000,70077
4,4,DWxIvQlpJK8,Metro Manila to be placed on lockdown due to c...,'Subscribe: https://www.youtube.com/@Rappler/\...,2020-03-12,104353,736,18.0,UCdnZdQxYXnbN4uWJg96oGxw,Rappler,2011-12-02,1930000,50275


In [11]:
df[['like_count', 'comment_count']].describe()

,like_count,comment_count
count,400.00000,397.000000
mean,487.52000,170.047859
std,1258.28274,428.701834
min,2.00000,0.000000
25%,53.00000,16.000000
50%,152.50000,54.000000
75%,386.00000,168.000000
max,12843.00000,6508.000000


In [12]:
view_count = np.array(df['view_count'])
like_count = np.array(df['like_count'])
comment_count = np.array(df['comment_count'])

In [13]:
comment_count

array([1.800e+01, 3.000e+00, 1.700e+01, 2.000e+00, 1.800e+01, 2.500e+01,
       3.000e+01, 3.000e+00, 6.190e+02, 8.100e+01, 4.000e+00, 1.310e+02,
       1.700e+01, 8.700e+01, 8.000e+01, 1.000e+01, 3.000e+00, 1.080e+03,
       9.500e+01, 1.680e+02, 5.320e+02, 1.369e+03, 4.170e+02, 9.480e+02,
       3.890e+02, 1.660e+02, 1.500e+01, 4.350e+02, 1.313e+03, 1.120e+02,
       1.860e+02, 6.600e+01, 2.200e+01, 9.400e+01, 3.000e+00, 2.600e+01,
       1.200e+01, 7.050e+02,       nan, 6.508e+03, 2.860e+02, 0.000e+00,
       1.120e+02, 5.800e+01, 1.100e+01, 5.300e+01,       nan, 1.289e+03,
       5.800e+01, 4.000e+00, 7.000e+01, 1.041e+03, 2.820e+02, 4.100e+01,
       0.000e+00, 5.500e+02, 3.000e+01, 3.200e+01, 2.110e+02, 1.740e+02,
       8.150e+02, 1.570e+02, 1.320e+02, 7.200e+01, 1.184e+03, 5.160e+02,
       5.000e+00, 3.200e+01, 2.300e+02, 9.300e+01, 4.500e+01, 1.520e+02,
       9.400e+01, 7.000e+00, 0.000e+00, 0.000e+00, 1.500e+01, 5.000e+00,
       4.740e+02, 6.010e+02, 1.350e+02, 5.200e+01, 

In [ ]:
norm_view_count = preprocessing.normalize([view_count])[0]
norm_like_count = preprocessing.normalize([like_count])[0]
norm_comment_count = preprocessing.normalize([comment_count])[0]